In [ ]:
bounds = data.rio.bounds()
print("Coordonnées projetées des limites :", bounds)
print(data.rio.crs)

easting = [bounds[0], bounds[2]]  # left, right
northing = [bounds[1], bounds[3]]  # bottom, top

# easting = [-16654000, 16654000]  # left, right
# northing = [-4000000, 1000000]  # bottom, top

# northing = [-2109501.7, 2109501.7]  # bottom, top

transformer = Transformer.from_crs(data.rio.crs, "EPSG:4326", always_xy=True)
# lon, lat = transformer.transform(easting, northing)
lon, lat = transformer.transform(easting, northing)

print(f"Coordonnées des coins en lat/lon :\nLatitude: {lat}\nLongitude: {lon}")

In [ ]:
import rioxarray
import pyproj

# Charger le GeoTIFF avec rioxarray
geo_tiff_path = lsf.load_file()
data = rioxarray.open_rasterio(geo_tiff_path)

# Vérifier la projection (CRS)
current_crs = data.rio.crs
print("Projection actuelle du fichier (CRS):", current_crs)

# Si le CRS n'est pas EPSG:4326, effectuer une conversion
if current_crs != "EPSG:4326":
    print("Le fichier n'est pas en EPSG:4326, conversion nécessaire.")
    
    # Transformer les coordonnées en EPSG:4326 (latitude/longitude)
    transformer = pyproj.Transformer.from_crs(current_crs, "EPSG:4326", always_xy=True)
    
    # Extraire les limites projetées du fichier (easting, northing)
    bounds = data.rio.bounds()
    print("Coordonnées projetées des limites :", bounds)

    # Coordonnées des limites (easting, northing)
    easting = [bounds[0], bounds[2]]  # left, right
    northing = [bounds[1], bounds[3]]  # bottom, top

    # Vérifier que les valeurs sont dans la plage de validité
    print(f"Vérification des valeurs easting: {easting}")
    print(f"Vérification des valeurs northing: {northing}")

    # Transformer les coordonnées projetées en lat/lon
    lon, lat = transformer.transform(easting, northing)
    
    # Afficher les coordonnées transformées (latitude, longitude)
    print(f"Coordonnées transformées en lat/lon :\nLatitude: {lat}\nLongitude: {lon}")
else:
    print("Le fichier est déjà en EPSG:4326 (latitude/longitude), aucune conversion nécessaire.")

In [ ]:
import rioxarray

# Charger le GeoTIFF avec rioxarray
geo_tiff_path = lsf.load_file()
data = rioxarray.open_rasterio(geo_tiff_path)

# Vérifier la projection (CRS)
current_crs = data.rio.crs
print("CRS complet (en WKT) :", current_crs)

# Extraire les informations du CRS sous forme de dictionnaire
crs_dict = current_crs.to_dict()
print(crs_dict)

# Vérifier le type de projection
projection_type = crs_dict.get('proj', None)
print("Type de projection :", projection_type)

# Test spécifique pour la projection Lambert Azimuthal Equal Area
if projection_type == 'laea':  # 'laea' correspond à Lambert Azimuthal Equal Area
    print("Le fichier utilise une projection Lambert Azimuthal Equal Area (LAEA).")
else:
    print(f"Projection détectée : {projection_type}")

In [ ]:
import rioxarray

# Charger le GeoTIFF avec rioxarray
geo_tiff_path = lsf.load_file()
data = rioxarray.open_rasterio(geo_tiff_path)

# Vérifier le CRS actuel
current_crs = data.rio.crs
print("CRS détecté :", current_crs)

# Si le CRS est incorrect, forcer l'assignation correcte (ESRI:54012 - Lambert Azimuthal Equal Area)
if current_crs != "ESRI:54012":
    print("Le fichier est mal référencé. Assignation correcte du CRS ESRI:54012.")

    # Réassigner le CRS correct sans reprojeter les données
    data = data.rio.write_crs("ESRI:54012", inplace=True)

    # Vérification
    print("Nouveau CRS après assignation :", data.rio.crs)
else:
    print("Le fichier a déjà le CRS correct.")

In [ ]:
# TEST CHATGPT


import rioxarray
from pyproj import CRS, Transformer

# Charger le GeoTIFF avec rioxarray
geo_tiff_path = lsf.load_file()
data = rioxarray.open_rasterio(geo_tiff_path)

# Vérifier le CRS actuel
current_crs = data.rio.crs
print("Projection actuelle du fichier (CRS):", current_crs)

# Vérification si le fichier est en EPSG:4326 (WGS84, lat/lon)
if current_crs != "EPSG:4326":
    print("Le fichier n'est pas en EPSG:4326, conversion nécessaire.")
    
    # Définir le CRS actuel du fichier (ESRI:54012) et le CRS cible (WGS84)
    crs_laea = CRS.from_string('ESRI:54012')  # CRS pour Lambert Azimuthal Equal Area
    crs_wgs84 = CRS.from_epsg(4326)  # CRS pour WGS84 (latitude/longitude)
    
    # Créer le transformer pour passer de ESRI:54012 à WGS84
    transformer_to_wgs84 = Transformer.from_crs(crs_laea, crs_wgs84, always_xy=True)
    
    # Extraire les limites projetées du fichier (easting, northing)
    bounds = data.rio.bounds()
    print("Coordonnées projetées des limites :", bounds)

    # Coordonnées des limites (easting, northing)
    easting = [bounds[0], bounds[2]]  # left, right (easting)
    northing = [bounds[1], bounds[3]]  # bottom, top (northing)

    # Transformer les coordonnées projetées en lat/lon
    lon, lat = transformer_to_wgs84.transform(easting, northing)
    
    # Afficher les coordonnées transformées (latitude, longitude)
    print(f"Coordonnées transformées en lat/lon :\nLatitude: {lat}\nLongitude: {lon}")
else:
    print("Le fichier est déjà en EPSG:4326 (latitude/longitude), aucune conversion nécessaire.")

In [ ]:
from pyproj import CRS, Transformer

# Définir le CRS pour Eckert IV
crs_eckert_iv = CRS.from_proj4('+proj=eck4 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs')
crs_wgs84 = CRS.from_epsg(4326)  # CRS pour WGS84 (latitude/longitude)

# Créer le transformer pour passer de Eckert IV à WGS84
transformer_to_wgs84 = Transformer.from_crs(crs_eckert_iv, crs_wgs84, always_xy=True)

# Exemple de coordonnées en Eckert IV (easting, northing)
easting = [-16654000, 8460000]  # Remplacez par vos valeurs
northing = [-1600000, 8460000]  # Remplacez par vos valeurs

# Transformer les coordonnées projetées en lat/lon
longitude, latitude = transformer_to_wgs84.transform(easting, northing)

# Afficher les coordonnées transformées (latitude, longitude)
print(f"Longitude: {longitude}\nLatitude: {latitude}")